In [12]:
import pandas as pd
import json
from sklearn.feature_selection import SelectKBest, f_classif, chi2
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split as split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
# Path to the JSON file
file_path = r"C:\Users\MSC\OneDrive - Fraunhofer Austria Research GmbH\Desktop\NLP\data\trans_prob_temp.csv"

# Reading the JSON file into a DataFrame

data = pd.read_csv(file_path)
data.head()  # Display the first few rows of the DataFrame

# Drop irrelevant columns
data = data.drop(columns=['pos_tags', 'syntax_tree'])

# Display the first few rows of the dataset
display(data.head())
print(data.info())


,is_sarcastic,headline,ratio_func_words,ratio_unique_pos,length,pos_repetitions,ratio_noun,ratio_verb,ratio_adj,ratio_adv,ratio_pron,ratio_propn,syntactic_depth,branching_factor,reversed_probability,average_surprisingness
0,1,thirtysomething scientists unveil doomsday clo...,0.125000,0.625000,8,1,0.500000,0.125000,0.125000,0.000000,0.000,0.125000,5,1.400000,0.289101,0.922017
1,0,dem rep. totally nails why congress is falling...,0.266667,0.600000,15,1,0.133333,0.133333,0.133333,0.066667,0.000,0.266667,6,2.800000,0.849506,0.977015
2,0,eat your veggies: 9 deliciously different recipes,0.125000,0.875000,8,0,0.250000,0.125000,0.125000,0.125000,0.125,0.000000,4,1.750000,0.135305,0.887056
3,1,inclement weather prevents liar from getting t...,0.250000,0.500000,8,1,0.375000,0.375000,0.000000,0.000000,0.000,0.000000,6,1.166667,0.805611,0.961821
4,1,mother comes pretty close to using word 'strea...,0.272727,0.454545,11,1,0.181818,0.272727,0.000000,0.272727,0.000,0.000000,6,2.000000,0.458575,0.873509


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   is_sarcastic            28619 non-null  int64  
 1   headline                28619 non-null  object 
 2   ratio_func_words        28619 non-null  float64
 3   ratio_unique_pos        28619 non-null  float64
 4   length                  28619 non-null  int64  
 5   pos_repetitions         28619 non-null  int64  
 6   ratio_noun              28619 non-null  float64
 7   ratio_verb              28619 non-null  float64
 8   ratio_adj               28619 non-null  float64
 9   ratio_adv               28619 non-null  float64
 10  ratio_pron              28619 non-null  float64
 11  ratio_propn             28619 non-null  float64
 12  syntactic_depth         28619 non-null  int64  
 13  branching_factor        28619 non-null  float64
 14  reversed_probability    28619 non-null

In [8]:
# Define features (X) and target (y)
X = data.iloc[:, 1:]  # Columns 4-17
y = data['is_sarcastic']

# Split the data into training, validation, and test sets
SEED = 42
train_headlines, other_headlines = split(data, test_size=0.3, random_state=SEED)  # 70% training
val_headlines, test_headlines = split(other_headlines, test_size=0.5, random_state=SEED)  # 15% validation, 15% test

# Prepare data for the Logistic Regression model
# Extract the corresponding feature values for train, validation, and test
X_train = X.loc[train_headlines.index]
y_train = y.loc[train_headlines.index]
X_val = X.loc[val_headlines.index]
y_val = y.loc[val_headlines.index]
X_test = X.loc[test_headlines.index]
y_test = y.loc[test_headlines.index]

In [11]:
synX = X.drop(columns=['headline'])

# 1. Filter Method: Univariate Statistical Test (ANOVA F-test)
select_k_best = SelectKBest(score_func=f_classif, k='all')
anova_scores = select_k_best.fit(synX, y).scores_

# 2. Wrapper Method: Recursive Feature Elimination (RFE) with Logistic Regression
log_model = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=log_model, n_features_to_select=5)  # Select top 5 features
rfe.fit(synX, y)
rfe_ranks = rfe.ranking_

# 3. Embedded Method: Feature Importance from Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(synX, y)
rf_importances = rf_model.feature_importances_

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale each method's scores to [0, 1]
anova_scaled = scaler.fit_transform(anova_scores.reshape(-1, 1)).flatten()
rfe_scaled = scaler.fit_transform((1 / rfe_ranks).reshape(-1, 1)).flatten()  # Inverse for ranking
rf_scaled = scaler.fit_transform(rf_importances.reshape(-1, 1)).flatten()

# Combine the scores into a DataFrame
feature_scores = {
    'ANOVA_F': anova_scaled,
    'RFE': rfe_scaled,
    'RandomForest': rf_scaled
}
importance_df = pd.DataFrame(feature_scores, index=synX.columns)

# Add an average score column
importance_df['Average_Score'] = importance_df.mean(axis=1)
importance_df = importance_df.sort_values(by='RFE', ascending=False)

print("Feature Importance Scores:")
print(importance_df)

# Select features based on average score threshold
selected_features = importance_df[importance_df['RFE'] > 0.05].index.tolist()
print("Selected Features:", selected_features)

# # Keep only selected features
# X_selected = X[selected_features]

# # Updated dataset with selected features and headlines
# data_transformed = pd.concat([data[['is_sarcastic', 'headline']], X_selected], axis=1)
# print("Transformed DataFrame:")
# print(data_transformed.head())


Feature Importance Scores:
                         ANOVA_F       RFE  RandomForest  Average_Score
ratio_func_words        1.000000  1.000000      0.800903       0.933634
ratio_unique_pos        0.365082  1.000000      0.557736       0.640939
ratio_verb              0.556911  1.000000      0.561078       0.705996
ratio_adv               0.276317  1.000000      0.232271       0.502862
ratio_pron              0.142221  1.000000      0.176568       0.439596
ratio_propn             0.145956  0.444444      0.392269       0.327556
ratio_adj               0.104327  0.259259      0.319761       0.227783
average_surprisingness  0.034888  0.166667      1.000000       0.400518
branching_factor        0.423827  0.111111      0.535848       0.356929
reversed_probability    0.000000  0.074074      0.985165       0.353080
ratio_noun              0.292966  0.047619      0.470016       0.270200
syntactic_depth         0.286037  0.027778      0.307704       0.207173
pos_repetitions         0.008487  0.0

In [14]:
def get_top_features(X_train, importance_df, method):
    """
    Returns the X data (including the headlines) that contains the top 5 syntactic features
    based on the specified feature selection method.
    
    Parameters:
    X_train (pd.DataFrame): The training data including headlines.
    importance_df (pd.DataFrame): DataFrame containing feature importance scores.
    method (str): The feature selection method ('ANOVA_F', 'RFE', 'RandomForest').
    
    Returns:
    pd.DataFrame: DataFrame containing the top 5 syntactic features and headlines.
    """
    if method not in importance_df.columns:
        raise ValueError(f"Method {method} not found in importance_df columns.")
    
    # Select the top 5 features based on the specified method
    top_features = importance_df.nlargest(5, method).index.tolist()
    
    # Include the 'headline' column
    top_features_with_headline = ['headline'] + top_features
    
    # Return the X data with the selected features
    return X_train[top_features_with_headline]


In [16]:

# Example usage:
top_features_df = get_top_features(X_train, importance_df, 'RandomForest')
print(top_features_df.head())

                                                headline  \
12170  american express to offer 5 months of paternit...   
28552  watch: dolphin knocks stand-up paddleboarder o...   
6883           man who enjoys thing informed he is wrong   
28387  jonathan lipnicki to star as young 'dark helme...   
12932  publicist worried kanye west's support of trum...   

       average_surprisingness  reversed_probability  ratio_func_words  \
12170                0.870541              0.185371          0.272727   
28552                0.927349              0.517047          0.363636   
6883                 0.981490              0.879098          0.125000   
28387                0.921614              0.301929          0.384615   
12932                0.808175              0.004917          0.272727   

       ratio_verb  ratio_unique_pos  
12170    0.272727          0.636364  
28552    0.272727          0.545455  
6883     0.250000          0.625000  
28387    0.153846          0.538462  
12932    0